## Prepping the training dataset for the regulation classifier

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
from pandas.io.json import json_normalize
import os

In [2]:
cwd = os.getcwd()
print (str(cwd)) 

C:\Users\alasseter\Documents\Projects\H4 dependent spouses\sandbox


In [3]:
docket = 'USCIS-2010-0017'

In [4]:
# Pull 1000 records from a single docket

def api_pull(docket):
    api_key = 'UrLmUbQnbxq0sQvnWhnlS3FXeWxeTBzpHRcPv4GK'
    max_pull= 1000
    url = 'https://api.data.gov:443/regulations/v3/documents.json?api_key='+str(api_key)+'&dktid='+str(docket)+'&rpp='+str(max_pull)
    response = requests.get(url).json()
    total_comments=response['totalNumRecords']
    df = json_normalize(response['documents'])
    df=df.dropna(subset=['postedDate'])
    df=df.dropna(subset=['commentText'])
    df['date']=pd.to_datetime(df['postedDate'])
    df['total_comments']=total_comments
    df['length']=df['commentText'].apply(len)
    df['duplicate']=df.duplicated('commentText')
#     df=df.drop(df[df.duplicated('commentText')==True].index) # remove dups
#     df=df.drop(df[(  (df['commentText'].str.contains('attach'))
#                    | (df['commentText'].str.contains('attached'))
#                    | (df['commentText'].str.contains('attachment'))
#                    | (df['commentText'].str.contains('Attach'))
#                    | (df['commentText'].str.contains('Attached'))
#                    | (df['commentText'].str.contains('Attachment'))
#                   ) 
#                   & (df['length']<100)].index)    # remove short attachment messages
#     df=df.sort_values(by='length', ascending=False)
    return df

# Examine a single docket.

In [5]:
df=api_pull(docket)

In [6]:
len(df)

1000

In [7]:
pd.options.display.max_colwidth = 200
df[(df['attachmentCount']==0)&(df['length']<50)]['commentText']

3            I support this rule. Please pass this rule.
6                                                   Yes!
95                             Yes.. I support this bill
144                                              I agree
159                                                 Do's
163     Yes. This would definitely help many H4 spouses.
197                 This is a great rule. I support this
219    I strongly support this move. Highly recommended.
227                        it should be in action asap\n
229                                 Please approve ASAP.
237                               Need this desperately!
306                                       Pass the rule.
315                       I strongly support this change
325                 I support to H4(dependents) to Work.
357                     Very good step. Excellent idea. 
366                   Strongly reccommend the EAD to h4s
390                 Please provide EAD to H4 dependents!
406                   I strongl

In [8]:
df.tail()

,agencyAcronym,allowLateComment,attachmentCount,commentDueDate,commentStartDate,commentText,docketId,docketTitle,docketType,documentId,...,documentType,numberOfCommentsReceived,openForComment,postedDate,rin,title,date,total_comments,length,duplicate
995,USCIS,False,0,2014-07-11T23:59:59-04:00,2014-05-12T00:00:00-04:00,"Please pass this regulation, It will boost American economy and It will be end all our nightmare of sitting at home since years though we are highly skilled and educated.",USCIS-2010-0017,Allowing Certain H-4 Dependent Spouses to Apply for Employment Authorization,Rulemaking,USCIS-2010-0017-2715,...,Public Submission,1,False,2014-05-15T00:00:00-04:00,1615-AB92,Comment Submitted by Srikanth G.,2014-05-15 04:00:00,12466,171,False
996,USCIS,False,0,2014-07-11T23:59:59-04:00,2014-05-12T00:00:00-04:00,Thanks for The department of homeland security on proposed rule to authorize H4 dependent sources of principal H-1B nonimmigrants under consideration of permanent residency in United States. This...,USCIS-2010-0017,Allowing Certain H-4 Dependent Spouses to Apply for Employment Authorization,Rulemaking,USCIS-2010-0017-1421,...,Public Submission,1,False,2014-05-14T00:00:00-04:00,1615-AB92,Comment Submitted by Benjamin Joseph,2014-05-14 04:00:00,12466,1644,False
997,USCIS,False,0,2014-07-11T23:59:59-04:00,2014-05-12T00:00:00-04:00,My spouse is non-resident immigrant on H4 status. \n\nShe is an Engineering graduate in Electrical and Electronics area. She worked in multiple technologies for 5 years before I came to USA. After...,USCIS-2010-0017,Allowing Certain H-4 Dependent Spouses to Apply for Employment Authorization,Rulemaking,USCIS-2010-0017-6605,...,Public Submission,1,False,2014-06-10T00:00:00-04:00,1615-AB92,Comment Submitted by Shivakumar Gnanavel,2014-06-10 04:00:00,12466,815,False
998,USCIS,False,0,2014-07-11T23:59:59-04:00,2014-05-12T00:00:00-04:00,I strongly support this bill since only deserving H4 will get their freedom of work for which they been waiting since long due to status for not allowed to work.,USCIS-2010-0017,Allowing Certain H-4 Dependent Spouses to Apply for Employment Authorization,Rulemaking,USCIS-2010-0017-10955,...,Public Submission,1,False,2014-07-14T00:00:00-04:00,1615-AB92,Comment Submitted by Shail Sawant,2014-07-14 04:00:00,12466,161,False
999,USCIS,False,0,2014-07-11T23:59:59-04:00,2014-05-12T00:00:00-04:00,I fully support and welcome this initiative by the USCIS as it will give provide an opportunity to for qualified spouses to work and use their talents in the industry rather than sitting idle at t...,USCIS-2010-0017,Allowing Certain H-4 Dependent Spouses to Apply for Employment Authorization,Rulemaking,USCIS-2010-0017-10898,...,Public Submission,1,False,2014-07-14T00:00:00-04:00,1615-AB92,Comment Submitted by SriVidya T.,2014-07-14 04:00:00,12466,286,False


# Compile all dockets

In [9]:
# export to csv
df.to_csv('H4_docket.csv', index=False)